In [8]:
import numpy as np
import pandas as pd
import pickle
import torch
import torch.nn as nn
import os
from copy import deepcopy

In [9]:
#load data
base_path = os.path.dirname(os.getcwd())

full_folder_path = os.path.join(base_path,"saved_files", "fake_data")
# data_df=pd.read_pickle(os.path.join(full_folder_path,"Aug7_1mil.pkl"))
with open(os.path.join(full_folder_path,"Aug7_1mil.pkl"), 'rb') as f:
    clean_data_dict = pickle.load(f)
print(clean_data_dict.keys())
print(len(clean_data_dict['2_phi']))

data_dict=deepcopy(clean_data_dict)


dict_keys(['event', 'genWeight', 'MET_phi', '1_phi', '1_genPartFlav', '2_phi', '2_genPartFlav', '3_phi', '3_genPartFlav', 'charge_1', 'charge_2', 'charge_3', 'pt_1', 'pt_2', 'pt_3', 'pt_MET', 'eta_1', 'eta_2', 'eta_3', 'mass_1', 'mass_2', 'mass_3', 'deltaphi_12', 'deltaphi_13', 'deltaphi_23', 'deltaphi_1MET', 'deltaphi_2MET', 'deltaphi_3MET', 'deltaphi_1(23)', 'deltaphi_2(13)', 'deltaphi_3(12)', 'deltaphi_MET(12)', 'deltaphi_MET(13)', 'deltaphi_MET(23)', 'deltaphi_1(2MET)', 'deltaphi_1(3MET)', 'deltaphi_2(1MET)', 'deltaphi_2(3MET)', 'deltaphi_3(1MET)', 'deltaphi_3(2MET)', 'deltaeta_12', 'deltaeta_13', 'deltaeta_23', 'deltaeta_1(23)', 'deltaeta_2(13)', 'deltaeta_3(12)', 'deltaR_12', 'deltaR_13', 'deltaR_23', 'deltaR_1(23)', 'deltaR_2(13)', 'deltaR_3(12)', 'pt_123', 'mt_12', 'mt_13', 'mt_23', 'mt_1MET', 'mt_2MET', 'mt_3MET', 'mt_1(23)', 'mt_2(13)', 'mt_3(12)', 'mt_MET(12)', 'mt_MET(13)', 'mt_MET(23)', 'mt_1(2MET)', 'mt_1(3MET)', 'mt_2(1MET)', 'mt_2(3MET)', 'mt_3(1MET)', 'mt_3(2MET)', 'ma

In [22]:
input_data_names_ordered = [
    ['MET_phi', 'pt_MET'], 
    ['1_phi', 'charge_1', 'pt_1', 'eta_1', 'mass_1'], 
    ['2_phi', 'charge_2', 'pt_2', 'eta_2', 'mass_2'], 
    ['3_phi', 'charge_3', 'pt_3', 'eta_3', 'mass_3']
]
input_data_particle_order = ['MET', '1', '2', '3']

pair_order = ["MET_1", "MET_2", "MET_3", "1_2", "1_3", "2_3"]
used_labels2 = [
    ['deltaphi_1MET', 'mt_1MET'], 
    ['deltaphi_2MET', 'mt_2MET'], 
    ['deltaphi_3MET', 'mt_3MET'], 
    ['deltaphi_12', 'deltaeta_12', 'deltaR_12', 'mt_12', 'norm_mt_12'], 
    ['deltaphi_13', 'deltaeta_13', 'deltaR_13', 'mt_13', 'norm_mt_13'], 
    ['deltaphi_23', 'deltaeta_23', 'deltaR_23', 'mt_23', 'norm_mt_23']
]
labels = {}
for edge, features in zip(pair_order, used_labels2):
    labels[edge] = {feature: data_dict[feature] for feature in features if feature in data_dict}

input_data = {}
for particle, features in zip(input_data_particle_order, input_data_names_ordered):
    input_data[particle] = {feature: data_dict[feature] for feature in features if feature in data_dict}
lepton_labels=['1_2', '1_3', '2_3']
lepton_dict= {key: labels[key] for key in lepton_labels}

print(lepton_dict.keys())
print(len(lepton_dict['1_2']['deltaphi_12']))
print(input_data.keys())
print(input_data['1'].keys())
print(len(input_data['1']['pt_1']))


dict_keys(['1_2', '1_3', '2_3'])
568554
dict_keys(['MET', '1', '2', '3'])
dict_keys(['1_phi', 'charge_1', 'pt_1', 'eta_1', 'mass_1'])
568554


In [ ]:
def get_order_for_pair(pair):

    orders = {
        '1_2': ['1', '2', '3', 'MET'],
        '1_3': ['1', '3', '2', 'MET'],
        '2_3': ['2', '3', '1', 'MET'],
    }
    return orders.get(pair, None)

def reorder_and_flatten(input_data_tensor, pair):

    order = get_order_for_pair(pair)
    if not order:
        raise ValueError("Invalid pair")

    order_indices = [input_data_particle_order.index(p) for p in order]
    reordered_tensor = input_data_tensor[:, order_indices, :]


    flattened_tensor = reordered_tensor.view(input_data_tensor.size(0), -1)

    return flattened_tensor

import torch.nn.functional as F

class ParticleDNN(nn.Module):
    def __init__(self, feature_per_particle, hidden_layer_sizes, num_kinematic_outputs):
        super(ParticleDNN, self).__init__()

        self.first_layers = nn.ModuleDict({
            '1_2': nn.Linear(3 * feature_per_particle, hidden_layer_sizes[0]),
            '1_3': nn.Linear(3 * feature_per_particle, hidden_layer_sizes[0]),
            '2_3': nn.Linear(3 * feature_per_particle, hidden_layer_sizes[0])
        })
        
        self.layers = nn.ModuleList()
        for i in range(len(hidden_layer_sizes) - 1):
            self.layers.append(nn.Linear(hidden_layer_sizes[i], hidden_layer_sizes[i + 1]))


        self.output_layer = nn.Linear(hidden_layer_sizes[-1], num_kinematic_outputs)
        
        self.relu = nn.ReLU()

    def forward(self, x):

        combined_output = []
        pairs = ['1_2', '1_3', '2_3']
        for pair in pairs:
            reordered_x = reorder_and_flatten(x, pair)
            out = self.first_layers[pair](reordered_x)
            combined_output.append(out)
        

        out = torch.cat(combined_output, dim=1)


        for i, layer in enumerate(self.layers):
            out = self.relu(layer(out))


        out = self.output_layer(out)

        return out



In [ ]:

num_features_per_particle = 5 
hidden_layer_sizes = [64, 32]  
num_kinematic_outputs = 5  


model = ParticleDNN(num_features_per_particle, hidden_layer_sizes, num_kinematic_outputs)


sample_data = torch.randn(100, 4, 5)


predictions = model(sample_data)


print(predictions.shape)  


In [ ]:
class DNN_flexible2(nn.Module):
    def __init__(self, input_vars, pair_inputdim, hidden_layer_sizes):
        super(DNN_flexible2, self).__init__()

        layer_sizes = [len(input_vars)-2] + hidden_layer_sizes + [1]

        self.layers = nn.ModuleList()
        for i in range(len(layer_sizes) - 1):
            self.layers.append(nn.Linear(layer_sizes[i], layer_sizes[i + 1]))
        
        self.relu = nn.ReLU()

    def forward(self, x):
        for i in range(len(self.layers) - 1):
            x = self.relu(self.layers[i](x))
        return x
    
class DNN_flexible3(nn.Module):
    def __init__(self, input_vars, hidden_layer_sizes, pair_input_dim):
        super(DNN_flexible3, self).__init__()

        
        layer_sizes = [len(input_vars)-2] + hidden_layer_sizes + [1]

        self.layers = nn.ModuleList()
        for i in range(len(layer_sizes) - 1):
            
            if i == 0:  
                input_dim = layer_sizes[i]
            else:  
                input_dim = layer_sizes[i] + pair_input_dim
                
            self.layers.append(nn.Linear(input_dim, layer_sizes[i + 1]))

        self.relu = nn.ReLU()

    def forward(self, x, pair_input):
        for i in range(len(self.layers)):
            x = self.layers[i](x)
            
            
            if i != 0 and i < len(self.layers) - 1:
                x = torch.cat((x, pair_input), dim=1)
                
            x = self.relu(x)

        return x

